This notebook is belong to [here]( https://www.kaggle.com/competitions/amex-default-prediction/discussion/329436).

In [1]:
import pandas

## Simple Feature Engineering

## More about Aggregations / Group Statistics

## Bruteforce Feature Engineering

## Stacking

In [2]:
# モデルたくさん作って予測値を線形回帰とかでくっつける。特徴量関係ないじゃん

## Pseudolabeling

In [ ]:
# モデル作ってテストデータを予測。予測した値を疑似ラベルとしてトレインデータにする。特徴量関係ないじゃん